In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# 1. 환경변수 로드
load_dotenv()

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
database = os.getenv('DB_NAME')

# 2. SQLAlchemy 엔진 생성
engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4"
)

# 3. 채널 데이터 불러오기 및 저장
channel_df = pd.read_csv("channels_by_keywords_최종.csv")

# DB에 저장
channel_df.to_sql(name='Channel', con=engine, if_exists='append', index=False)
print("Channel 테이블에 데이터 삽입 완료")

# 4. 영상 데이터 불러오기
video_df = pd.read_csv("videos_by_keywords_최종.csv", encoding='utf-8')

# segment 컬럼 제거 (있다면)
if 'segment' in video_df.columns:
    video_df = video_df.drop(columns=['segment'])

# uploadDate 포맷 변환
video_df['uploadDate'] = pd.to_datetime(video_df['uploadDate'], utc=True, errors='coerce')
video_df['uploadDate'] = video_df['uploadDate'].dt.strftime('%Y-%m-%d %H:%M:%S')

# 컬럼명 정제
video_df.columns = ['videoID', 'channelID', 'title', 'viewCount', 'likeCount', 'commentCount',
                    'uploadDate', 'duration', 'tags', 'thumbnailURL', 'keyword', 'categoryID']

# 5. 채널 ID 유효한 것만 필터링
video_df = video_df.drop_duplicates(subset='videoID')
with engine.connect() as conn:
    valid_channels = pd.read_sql("SELECT channelID FROM Channel", conn)
valid_channel_ids = set(valid_channels['channelID'])

video_df = video_df[video_df['channelID'].isin(valid_channel_ids)]
print(f"삽입 가능한 영상 수: {len(video_df)}")

# 6. DB에 저장
video_df.to_sql(name='Video', con=engine, if_exists='append', index=False)
print("Video 테이블에 데이터 삽입 완료")

C:\Users\USER\AppData\Local\Temp\ipykernel_21196\642515264.py:25: UserWarning: The provided table name 'Channel' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  channel_df.to_sql(name='Channel', con=engine, if_exists='append', index=False)


Channel 테이블에 데이터 삽입 완료
삽입 가능한 영상 수: 103115
Video 테이블에 데이터 삽입 완료


C:\Users\USER\AppData\Local\Temp\ipykernel_21196\642515264.py:53: UserWarning: The provided table name 'Video' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  video_df.to_sql(name='Video', con=engine, if_exists='append', index=False)


In [3]:
# 1. channel 테이블에서 현재 존재하는 channelID 불러오기
valid_channel_ids = pd.read_sql("SELECT channelID FROM channel", con=engine)
valid_ids_set = set(valid_channel_ids['channelID'])

# 2. video_df에서 존재하지 않는 channelID만 필터링
invalid_channel_df = video_df[~video_df['channelID'].isin(valid_ids_set)]

# 3. 결과 확인
print(f"존재하지 않는 channelID 수: {invalid_channel_df['channelID'].nunique()}")
print(f"해당되는 영상 수: {len(invalid_channel_df)}")

존재하지 않는 channelID 수: 36
해당되는 영상 수: 64


In [5]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
from dotenv import load_dotenv
import os

# 환경 변수 불러오기
load_dotenv()
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
database = os.getenv('DB_NAME')

# DB 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4")

# CSV 파일 로드
pos_df = pd.read_csv('pos_tagged_data.csv')

# 컬럼 확인
print(pos_df.columns)

# videoID 기준 중복 제거
pos_df = pos_df.drop_duplicates(subset='videoID')

# DB 연결 후 업데이트
with engine.begin() as conn:
    for _, row in pos_df.iterrows():
        video_id = row['videoID']
        pos_tags = row['pos_tags']

        # SQLAlchemy text 쿼리 + 딕셔너리 파라미터
        conn.execute(
            text("""
                UPDATE video
                SET token_tagging = :token_tagging
                WHERE videoID = :videoID
            """),
            {"token_tagging": pos_tags, "videoID": video_id}
        )

print("video 테이블에 token_tagging 업데이트 완료.")

Index(['videoID', 'channelID', 'title', 'viewCount', 'likeCount',
       'commentCount', 'uploadDate', 'duration', 'tags', 'thumbnailURL',
       'keyword', 'segment', 'categoryID', 'text_data', 'pos_tags'],
      dtype='object')
video 테이블에 token_tagging 업데이트 완료.
